In [37]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
import xgboost as xgb
from scipy.stats.mstats import winsorize
from sklearn.metrics import f1_score as score #Scoring metric for the competition
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import time
import warnings
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
sns.set_palette("Set2")
sns.set_style("ticks")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


**Import Data**

In [ ]:
# prompt: from drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Load the data
def load_data():
    train_values = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini_competition/train_values.csv")
    train_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini_competition/train_labels.csv")
    test_values = pd.read_csv(("/content/drive/MyDrive/Colab Notebooks/mini_competition/test_values.csv"))
    return train_values, train_labels, test_values

In [39]:
# Call the load_data function to define the variables
train_values, train_labels, test_values = load_data()

**Checking Data**

In [40]:
print('Train Values : {}'.format(train_values.shape))
print('Train Labels : {}'.format(train_labels.shape))
print('Test Values : {}'.format(test_values.shape))

Train Values : (260601, 39)
Train Labels : (260601, 2)
Test Values : (86868, 39)


In [46]:
train_labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [43]:
train_values.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [45]:
test_values.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


**Data Preparation**

Merge Feature and Label on Training Data

**Main File**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


**Load data**

**# Step 2: Preprocess the data**

In [48]:
# Step 2: Preprocess the data
def preprocess_data(train_values, train_labels, test_values):
    # Merge train values and labels
    train_data = pd.merge(train_values, train_labels, on='building_id')
    return train_data



In [49]:
# Preprocess the data
train_data = preprocess_data(train_values, train_labels, test_values)

In [50]:
train_data.select_dtypes(include=['int64']).columns

Index(['building_id', 'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'damage_grade'],
      dt

In [51]:
train_data.select_dtypes(include=['object']).columns

Index(['land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'legal_ownership_status'],
      dtype='object')

In [52]:
# Define feature types
numeric_features = ['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families']
categorical_features = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']

In [53]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Create preprocessing steps
numeric_transformer = StandardScaler() #Removed extra space at the beginning of the line
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [54]:
# prompt: def preprocess_data(train_data, test_values): with return values
#     # Separate features and target
#     X = train_data.drop(['building_id', 'damage_grade'], axis=1)
#     y = train_data['damage_grade']
# generate next code
#     # Split into training and validation sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def split_data(train_data, test_values):
    # Separate features and target
    X = train_data.drop(['building_id', 'damage_grade'], axis=1)
    y = train_data['damage_grade']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, y_test, test_values

# split the data
X_train, X_test, y_train, y_test, y_test, test_values = split_data(train_data, test_values)







**Step 3: Train the model**

In [55]:
# prompt: generate pipeline to create random forest

from sklearn.pipeline import Pipeline

def create_pipeline():
  pipeline = Pipeline([
      ('preprocessor', preprocessor),
      ('random_forest', RandomForestClassifier(n_estimators=100, random_state=42))
  ])
  return pipeline

pipeline = create_pipeline()

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the testing data
y_pred = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# You can now use this pipeline to fit the model on the entire training data and predict on the test data


Accuracy: 0.5646476468218184

Classification Report:
              precision    recall  f1-score   support

           1       0.50      0.33      0.40      5170
           2       0.60      0.75      0.67     29487
           3       0.47      0.33      0.38     17464

    accuracy                           0.56     52121
   macro avg       0.52      0.47      0.48     52121
weighted avg       0.55      0.56      0.55     52121



**XGBOOST**

In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
import xgboost as xgb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
# Load the data
def load_data():
    train_values = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini_competition/train_values.csv")
    train_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini_competition/train_labels.csv")
    test_values = pd.read_csv(("/content/drive/MyDrive/Colab Notebooks/mini_competition/test_values.csv"))
    return train_values, train_labels, test_values

In [58]:
def preprocess_data(train_values, train_labels, test_values):
    # Merge train values and labels
    train_data = pd.merge(train_values, train_labels, on='building_id')

    # Identify numeric and categorical features
    # You may need to adjust these based on your specific dataset
    numeric_features = train_data.select_dtypes(include=['int64', 'float64']).columns.tolist()
    numeric_features = [col for col in numeric_features if col not in ['building_id', 'damage_grade']]
    categorical_features = train_data.select_dtypes(include=['object']).columns.tolist()

    # Create preprocessing steps
    numeric_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ])

    # Prepare the data
    X = train_data.drop(['building_id', 'damage_grade'], axis=1)
    y = train_data['damage_grade']
    X_test = test_values.drop('building_id', axis=1)

    # Adjust labels to start from 0
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    # Split the training data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, preprocessor

In [59]:
# Train XGBoost model
def train_xgboost(X_train, y_train, X_val, y_val, preprocessor):
    model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42))
    ])

    model.fit(X_train, y_train)

    return model

In [63]:
# Make predictions
def make_predictions(model, X):
    return model.predict(X)

In [67]:
# Evaluate model
def evaluate_model(y_true, y_pred):
    print("Classification Report:")
    print(classification_report(y_true, y_pred))

    f1 = f1_score(y_true, y_pred, average='weighted')
    print(f"Weighted F1 Score: {f1:.4f}")



In [68]:
def main():
    # Load data
    train_values, train_labels, test_values = load_data()

    # Preprocess data
    X_train, X_val, X_test, y_train, y_val, preprocessor = preprocess_data(train_values, train_labels, test_values)

    # Train model
    model = train_xgboost(X_train, y_train, X_val, y_val, preprocessor)

    # Make predictions on validation set
    val_predictions = make_predictions(model, X_val)

    # Evaluate model
    print("Validation Set Evaluation:")
    evaluate_model(y_val, val_predictions)

    # Make predictions on test data
    test_predictions = make_predictions(model, X_test)

    # You can save or further process the test predictions here

if __name__ == "__main__":
    main()

Validation Set Evaluation:
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.38      0.49      5170
           1       0.69      0.86      0.76     29487
           2       0.72      0.51      0.60     17464

    accuracy                           0.69     52121
   macro avg       0.70      0.59      0.62     52121
weighted avg       0.70      0.69      0.68     52121

Weighted F1 Score: 0.6812


In [72]:
def main():
    # Load data
    train_values, train_labels, test_values = load_data()

    # Preprocess data
    X_train, X_val, X_test, y_train, y_val, preprocessor = preprocess_data(train_values, train_labels, test_values)

    # Train model
    model = train_xgboost(X_train, y_train, X_val, y_val, preprocessor)

    # Make predictions on validation set
    val_predictions = make_predictions(model, X_val)

    # Evaluate model
    print("Validation Set Evaluation:")
    evaluate_model(y_val, val_predictions)

    # Make predictions on test data
    test_predictions = make_predictions(model, X_test)

    # You can save or further process the test predictions here

    submission_df = pd.DataFrame({'building_id': test_values['building_id'], 'damage_grade': test_predictions})
    submission_df.to_csv('submission.csv', index=False)

if __name__ == "__main__":
    main()

Validation Set Evaluation:
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.38      0.49      5170
           1       0.69      0.86      0.76     29487
           2       0.72      0.51      0.60     17464

    accuracy                           0.69     52121
   macro avg       0.70      0.59      0.62     52121
weighted avg       0.70      0.69      0.68     52121

Weighted F1 Score: 0.6812
